<a href="https://colab.research.google.com/github/harshithamadarapu/Team16_Hinglish-Auto-suggestions/blob/main/FineTuneing(indic%20bert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Path to the Excel file
excel_file = '/content/small.xlsx'

# Path to save the CSV file
csv_file = '/content/data.csv'

# Read the Excel file
data = pd.read_excel(excel_file, engine='openpyxl')

# Save it as a CSV file
data.to_csv(csv_file, index=False)

print(f"Excel file '{excel_file}' has been converted to CSV as '{csv_file}'.")


Excel file '/content/small.xlsx' has been converted to CSV as '/content/data.csv'.


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Load pre-trained IndicBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModelForMaskedLM.from_pretrained('ai4bharat/indic-bert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

In [3]:
class HinglishDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_length=128):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        encoding = self.tokenizer(sentence, padding='max_length', truncation=True,
                                   max_length=self.max_length, return_tensors='pt')

        # Ensure both input_ids and attention_mask are returned
        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # Remove batch dimension
            'attention_mask': encoding['attention_mask'].squeeze(0)  # Remove batch dimension
        }


In [4]:
# Load your dataset (use the correct path to your file)
df = pd.read_csv('/content/data.csv')
sentences = df['phrases'].tolist()

# Create the dataset and dataloader
dataset = HinglishDataset(sentences, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [6]:
import torch
from torch.optim import AdamW


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fine-tuning loop
model.train()  # Set model to training mode
for epoch in range(2):  # Adjust the number of epochs as needed
    total_loss = 0
    for batch in dataloader:
        # Move batch to device (GPU/CPU)
        batch = {key: value.to(device) for key, value in batch.items()}


        # Forward pass
        outputs = model(input_ids=batch['input_ids'],
                        attention_mask=batch['attention_mask'],
                        labels=batch['input_ids'])  # For language models, the labels are usually the same as input_ids

        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()

    # Print average loss per epoch
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")


Epoch 1, Loss: 18.44899207977295
Epoch 2, Loss: 18.448992131347655
